In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger
from keras.models import load_model

# Define the directories for the model and training data sets
model_dir = "./cartoon_character_classifier.h5"
train_dir = "/content/train_data"

# Define the image size, batch size and number of classes
img_size = (64, 64)
batch_size = 32
class_num = len(os.listdir(train_dir))

# Make a logger
csv_logger = CSVLogger("model_history_log.csv", append=True)

In [ ]:
# Create an ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.2
)

# Create the training and validation generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    color_mode="grayscale",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    color_mode="grayscale",
    subset="validation"
)

#  Load the classification model, otherwise build a new model
if os.path.exists(model_dir):
    
    model = load_model(model_dir)

else:

    # Define the model architecture
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(64, 64, 1)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(class_num, activation="softmax")
    ])

    # Compile the model
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0008),
        metrics=["accuracy"]
    )


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    batch_size = batch_size,
    validation_data=val_generator,
    callbacks=[csv_logger]
)

# Save the model
model.save("cartoon_character_classifier.h5")


In [ ]:
# Read the CSV file into a pandas dataframe
df = pd.read_csv('model_history_log.csv')

# Plot the accuracy and loss graphs for each epoch
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

ax1.plot(df['accuracy'])
ax1.plot(df['val_accuracy'])
ax1.set_title('Model accuracy')
ax1.set_ylabel('Accuracy')
ax1.set_xlabel('Epoch')
ax1.legend(['Train', 'Validation'], loc='upper left')
ax1.set_ylim([0, 1])

ax2.plot(df['loss'])
ax2.plot(df['val_loss'])
ax2.set_title('Model loss')
ax2.set_ylabel('Loss')
ax2.set_xlabel('Epoch')
ax2.legend(['Train', 'Validation'], loc='upper left')
ax2.set_ylim(bottom=0)

plt.show()


In [ ]:
# Report training, validation data set count of each class
train_folders = os.listdir(train_dir)

data = []
columns = ["label", "name", "train_count", "val_count", "total_count"]
i = 0

for folder_name in train_folders:
    folder_dir = os.path.join(train_dir, folder_name)
    img_count = len(os.listdir(folder_dir))
    train_count = int(img_count * 0.8)
    val_count = img_count - train_count

    data.append([i, folder_name, train_count, val_count, img_count])
    i += 1

pd.set_option('display.width', 2000)
pd.set_option('display.max_rows', None)
df = pd.DataFrame(data, columns=columns)
print(df)